We're want to find out not just which hyperparameters are ideal, but which are most probable to get us to meet the submission threshold (<0.09 loss on the training set, <0.18 loss on the validation set.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = 'hyperparameter_tests/runs_detail.csv'
#data_path = 'hyperparameter_tests/runs_12node.csv'
#data_path = 'hyperparameter_tests/runs_988lr.csv'


run_data = pd.read_csv(data_path)

In [3]:
run_data.head()

,learning_rate,hidden_nodes,100,200,300,400,500,600,700,800,...,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000
0,0.8,5,0.569052,0.451932,0.437519,0.427405,0.426247,0.412109,0.442707,0.468199,...,0.158143,0.158520,0.156635,0.161255,0.243910,0.172350,0.202368,0.170503,0.171274,0.141295
1,0.8,5,0.548160,0.459000,0.504979,0.415933,0.398892,0.385612,0.378796,0.372313,...,0.215934,0.185130,0.449059,0.187484,0.174877,0.195310,0.231660,0.179763,0.265720,0.198976
2,0.8,5,0.634337,0.484782,0.459772,0.432097,0.417385,0.447732,0.388189,0.382954,...,0.177569,0.197886,0.250602,0.175986,0.166781,0.182495,0.169448,0.166634,0.163998,0.232034
3,0.8,5,0.702802,0.465733,0.450884,0.433184,0.424518,0.399189,0.394031,0.449217,...,0.296645,0.251792,0.225737,0.221706,0.183848,0.187251,0.199833,0.181737,0.288228,0.205039
4,0.8,5,0.564931,0.449616,0.437521,0.418159,0.419728,0.421942,0.400075,0.414640,...,0.186072,0.181086,0.170219,0.160090,0.159073,0.153786,0.158940,0.147201,0.161364,0.146609


In [4]:
def first_passing(series, threshold=0.18):
    print(series)
    for i, value in enumerate(series):
        if value < threshold:
            return i+1
    return np.nan

In [14]:
from collections import defaultdict

# access order: number of nodes, learning rate, iteration count (list of experiments)
data = defaultdict(lambda : defaultdict(lambda: defaultdict(list)))

for i, row in run_data.iterrows():
    frame = data[int(row.get('hidden_nodes'))][row.get('learning_rate')]
    for j in range(100, 3001, 100):
        frame[j].append(row.get(str(j)))

In [15]:
def percentage_below(series, threshold=0.18):
    return sum(1 for s in series if s < threshold) / len(series)

In [33]:
table = defaultdict(dict)
iter_stop = 2000
threshold = 0.18

for num_nodes, rest in data.items():
    for learning_rate, series in rest.items():
        table[num_nodes][learning_rate] = percentage_below(series[iter_stop], threshold)
        #table[num_nodes][learning_rate] = pd.Series(series[iter_stop]).quantile(0.5)
        
pdt = pd.DataFrame(table)
pdt

,5,6,7,8,9,10,11,12,13,14
0.80,0.12500,0.28125,0.40625,0.31250,0.28125,0.46875,0.53125,0.43750,0.65625,0.56250
0.83,0.25000,0.40625,0.40625,0.31250,0.59375,0.56250,0.62500,0.65625,0.62500,0.81250
0.86,0.25000,0.31250,0.31250,0.62500,0.53125,0.53125,0.78125,0.90625,0.75000,0.75000
0.89,0.28125,0.46875,0.50000,0.68750,0.71875,0.65625,0.75000,0.75000,0.84375,0.84375
0.92,0.31250,0.56250,0.46875,0.62500,0.68750,0.71875,0.62500,0.90625,0.78125,0.75000
0.95,0.40625,0.56250,0.56250,0.84375,0.78125,0.75000,0.75000,0.93750,0.87500,0.68750
0.98,0.40625,0.53125,0.78125,0.90625,0.75000,0.84375,0.75000,0.87500,0.78125,0.59375
1.01,0.50000,0.46875,0.78125,0.90625,0.84375,0.87500,0.78125,0.78125,0.84375,0.62500
1.04,0.59375,0.81250,0.84375,0.84375,0.68750,0.93750,0.78125,0.71875,0.87500,0.53125
1.07,0.56250,0.71875,0.78125,0.68750,0.90625,0.84375,0.96875,0.87500,0.68750,0.46875


In [8]:
pdt.idxmax(axis=0)

5     1.37
6     1.40
7     1.40
8     1.31
9     1.22
10    1.10
11    1.19
12    1.07
13    1.01
14    1.01
dtype: float64

In [9]:
pdt.idxmax(axis=1)

0.80    12
0.83     6
0.86    11
0.89    14
0.92    11
0.95    12
0.98    13
1.01    10
1.04    11
1.07    12
1.10    10
1.13     9
1.16     9
1.19     7
1.22     9
1.25     9
1.28     6
1.31     8
1.34     9
1.37     5
1.40     6
dtype: int64